In [11]:
import pandas as pd
from ast import literal_eval

generic = lambda x: literal_eval(x)
conv = {'nutrition' : generic, 'steps' : generic, 'ingredients' : generic, 'id_column' : generic, 'jaccard_similarity' : generic, 'diff' : generic, 'recipes' : generic, 'ingredients_original' : generic, 'tags' : generic}
df = pd.read_csv(r"C:\Users\01din\PycharmProjects\thesis\data\cleaned_recipes\recipes_with_JS.csv", converters=conv)
df.drop(['Unnamed: 0'], inplace=True, axis=1)

This was the w2v model that was used to compute the semantic similarity betweeen recipes. After preprocessing, it took all textual entities of the recipe and concatenated them with seperator tokens in between.

In [12]:
#Punctuation, lowercasing, etc.
def preprocess_text(text):
    text = str(text)
    text = text.lower()
    text = ''.join([c for c in text if c not in ('!', '.', ':', ',', ';', '?', '(', ')', '[', ']', '{', '}')])
    return text

#For things like the steps column, which contained lists of strings
def preprocess_list(lst):
    return ' '.join([preprocess_text(item) for item in lst])

#Applying the methods
df['combined'] = (
    '<name> ' + df['name'].apply(preprocess_text) + ' '
    '<ingredients> ' + df['ingredients_original'].apply(preprocess_list) + ' '
    '<steps> ' + df['steps'].apply(preprocess_list) + ' '
    '<tags> ' + df['tags'].apply(preprocess_list) + ' '
    '<description> ' + df['description'].apply(preprocess_text)
)

In [15]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

#Tokenize the combined methods with gensims simple_preprocess
df['tokenized'] = df['combined'].apply(simple_preprocess)

#Make list to train w2v model
documents = df['tokenized'].tolist()
model = Word2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)

model.save(r"C:\Users\01din\PycharmProjects\thesis\models\w2v\w2v4js.model")

In [16]:
df.to_csv(r"C:\Users\01din\PycharmProjects\thesis\data\cleaned_recipes\recipes_with_JS.csv")